In [1]:
import cv2
import numpy as np
import os
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, Activation, ZeroPadding2D, Dense, MaxPooling2D, GlobalAveragePooling2D, Dropout, Input, merge, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model
import keras.optimizers as optimizers

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
datagen = ImageDataGenerator(rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
            )

In [3]:
train = datagen.flow_from_directory('/input/invasive-species/train',
                                              target_size = (224, 224),
                                              batch_size = 16,
                                              class_mode = 'categorical',
                                              
                                              )
validation = datagen.flow_from_directory('/input/invasive-species/test',
                                              target_size = (224, 224),
                                              batch_size = 16,
                                              class_mode = 'categorical'
                                              )

Found 34110 images belonging to 2 classes.
Found 208 images belonging to 2 classes.


No handlers could be found for logger "deepsense.neptune.common.websockets.websocket_client_adapter"


In [4]:
base_model = ResNet50( weights='imagenet', include_top=False)

92545024/94653016 [============================>.] - ETA: 0s

In [74]:
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_7 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, None, None, 3) 0           input_7[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, None, None, 64 9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, None, None, 64 256         conv1[0][0]                      
___________________________________________________________________________________________

In [5]:
base_model.output

<tf.Tensor 'avg_pool/AvgPool:0' shape=(?, ?, ?, 2048) dtype=float32>

In [5]:
#base_model.summary()
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax')(x)
new_model = Model(inputs=base_model.input, outputs= x)

In [10]:
for layer in base_model.layers[:]:
    layer.trainable = True

In [17]:
#optimizer = optimizers.Adam(lr=0.001)
optimizer=optimizers.SGD(lr=0.00000001, decay=1e-6, momentum=0.9, nesterov=True)
new_model.compile(loss='categorical_crossentropy',
          optimizer=optimizer,
          metrics=['accuracy'])

In [18]:
checkpoint = ModelCheckpoint('species_2.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [19]:
new_model.fit_generator(train, steps_per_epoch = 2132, validation_data=validation, validation_steps=13, epochs=4,  callbacks=[checkpoint])

Epoch 1/4
2132/2132 [==============================] - 1308s - loss: 0.0205 - acc: 0.9934 - val_loss: 0.0506 - val_acc: 0.9808
Epoch 2/4
2132/2132 [==============================] - 1302s - loss: 0.0216 - acc: 0.9934 - val_loss: 0.0565 - val_acc: 0.9904
Epoch 3/4
2132/2132 [==============================] - 1331s - loss: 0.0222 - acc: 0.9932 - val_loss: 0.0713 - val_acc: 0.9808
Epoch 4/4
  18/2132 [..............................] - ETA: 1288s - loss: 0.0213 - acc: 0.9931

KeyboardInterrupt: 

In [50]:
new_model.save_weights()